## Practice Questions & Answers

### Q1: Why is Amazon S3 considered Object Storage instead of a file system?


S3 is object storage, not a file system, due to these fundamental differences:

| Aspect | File System | S3 Object Storage |
|--------|-------------|-------------------|
| **Structure** | Hierarchical with directories, inodes | Flat namespace with key-value pairs |
| **Operations** | Random read/write, seek, append | Only GET/PUT entire objects |
| **Metadata** | Limited (permissions, timestamps) | Rich, custom metadata support |
| **Modification** | In-place updates possible | Must replace entire object |
| **Locking** | File locks supported | No native locking |
| **Path** | Real directory traversal | "Folders" are just key prefixes |

**Key Point:** You cannot open a file in S3, seek to byte 1000, and modify 10 bytes. You must download the entire object, modify it locally, and upload the new version.





### Q2: Explain the relationship between Bucket, Object, and Key in S3.

In Amazon S3, a bucket is a top-level container, an object is the stored data inside bucket, and the key is the unique identifier that locates the object within the bucket. There is no real directories, only keys with prefixes.

1️. Bucket   
  - A bucket is the top-level container in S3
  - It lives in a single AWS Region
  - Bucket names are globally unique
  - Used for:
    - Organizing data
    - Applying policies (bucket policy, access points)
    - Managing settings (versioning, lifecycle, encryption)

  > 📦 Think of a bucket as a root namespace

2️. Object  
  - An object is the actual data stored in S3
  - Each object consists of:
    - Data (file contents)
    - Metadata
    - Key
  - Objects are immutable (updates = new object)

  > 📄 Objects are the units of storage

3️. Key    
  - A key is the unique identifier of an object within a bucket
  - It is a string, not a real folder path
  - “Folders” in S3 are logical, created by key prefixes

  > 🔑 Key = object’s full path name



### Q3: Why can't you update a single row inside a CSV file stored in S3?

S3 is **immutable object storage**. When you store a CSV file, it becomes a single binary blob (object). S3 has no understanding of:
- Row structure
- CSV format
- Line boundaries

**To modify one row, you must:**
1. Download the entire CSV file
2. Parse it locally (e.g., with pandas)
3. Modify the specific row
4. Upload the complete new file
5. The new upload replaces the old object (or creates a new version)

**This is why data lakes often use:**
- **Parquet** format with partitioning for efficient updates
- **Delta Lake / Iceberg** for ACID transactions on S3
- **Append-only patterns** rather than updates



### Q4: What is an S3 prefix, and why is it important for Spark/Hive performance?


A **prefix** is the "folder-like" portion of an object key before the actual filename.

**Why it matters for performance:**

```text
Without Prefix Partitioning:
Query: SELECT * FROM sales WHERE date = '2024-01-15'
→ Spark must scan ALL files: s3://bucket/*.parquet
→ Full table scan!

With Prefix Partitioning:
s3://bucket/year=2024/month=01/day=15/*.parquet
Query: SELECT * FROM sales WHERE date = '2024-01-15'
→ Spark only reads: s3://bucket/year=2024/month=01/day=15/
→ Partition pruning! Only relevant data scanned.
```

**Performance Benefits:**
1. **Partition Pruning**: Read only relevant "directories"
2. **Parallelism**: Spark reads different prefixes in parallel
3. **Reduced I/O**: Less data transferred from S3
4. **Cost Savings**: Fewer S3 GET requests


### Q5: Compare IAM Policy, Bucket Policy, and Access Point Policy.


| Comparison | IAM Policy | Bucket Policy | Access Point Policy |
|------------|-----------|---------------|---------------------|
| **Attachment** | IAM user/role/group | S3 bucket | S3 access point |
| **Scope** | What this identity can do | Who can access this bucket | Who can access via this AP |
| **Principal** | Implicit (attached entity) | Explicit (specified in policy) | Explicit (specified in policy) |
| **Cross-Account** | Requires bucket policy too | Can grant alone | Can grant alone |
| **Scale** | Per-user management | Single policy per bucket | One policy per access point |
| **Best For** | Internal users | Public access, compliance | Large-scale, multi-team |

**Access Decision:**
```text
Access Granted IF:
(IAM Policy ALLOWS) AND (Bucket/AP Policy ALLOWS) AND (No Explicit DENY)
```



### Q6: What problem does S3 Versioning solve, and what new problem does it introduce?


**Problems SOLVED:**

| Problem | How Versioning Solves It |
|---------|-------------------------|
| Accidental deletion | Delete adds marker, original versions preserved |
| Accidental overwrite | Previous versions retained |
| Ransomware | Malicious encryption creates new version, originals safe |
| Audit requirements | Version history provides change tracking |
| Recovery | Can restore any previous version |

**Problems INTRODUCED:**

| New Problem | Impact | Mitigation |
|-------------|--------|------------|
| Increased storage costs | All versions stored and billed | Lifecycle rules to expire old versions |
| Management complexity | Many versions to track | S3 Inventory to audit |
| Delete marker accumulation | Clutters namespace | Lifecycle rule to expire delete markers |
| Replication complexity | Must handle version IDs | Configure replication properly |




### Q7: What is the difference between SRR and CRR in S3 replication?



| Aspect | SRR (Same-Region Replication) | CRR (Cross-Region Replication) |
|--------|-------------------------------|--------------------------------|
| **Geography** | Source and destination in SAME region | Source and destination in DIFFERENT regions |
| **Latency** | Lower (same region) | Higher (cross-region transfer) |
| **Cost** | No data transfer charges | Inter-region data transfer charges |
| **DR Scope** | Single region failures only | Survives entire region failure |
| **Use Cases** | Log aggregation, compliance copies, test environments | Disaster recovery, global access, compliance |
| **Network** | Uses AWS backbone, same region | Uses AWS global network |

**Common Requirements for Both:**
- Versioning must be enabled on both buckets
- Proper IAM permissions configured
- Bucket owner must have access to both buckets

---



# SQL Coding Exercise

## Problem Statement

Given the `orders` table:
```sql
orders (
    order_id STRING,
    user_id STRING,
    order_date DATE,
    amount DOUBLE,
    country STRING
)
```

**Write a query to calculate::**
For each country in 2024   
1. Find users who placed at least 3 orders
2. Calculate total order amount from those qualified users

**Expected Output:**
```text
country | qualified_user_count | total_amount
```

## Solution

```sql
-- Solution using CTE (Common Table Expression)
WITH qualified_users AS (
    -- Step 1: Find users with at least 3 orders per country in 2024
    SELECT 
        country,
        user_id,
        COUNT(*) AS order_count,
        SUM(amount) AS user_total_amount
    FROM orders
    WHERE YEAR(order_date) = 2024
    GROUP BY country, user_id
    HAVING COUNT(*) >= 3
)
-- Step 2: Aggregate by country
SELECT 
    country,
    COUNT(DISTINCT user_id) AS qualified_user_count,
    SUM(user_total_amount) AS total_amount
FROM qualified_users
GROUP BY country
ORDER BY country;
```

## Alternative Solution (Subquery)

```sql
SELECT 
    country,
    COUNT(*) AS qualified_user_count,
    SUM(user_amount) AS total_amount
FROM (
    SELECT 
        country,
        user_id,
        SUM(amount) AS user_amount
    FROM orders
    WHERE YEAR(order_date) = 2024
    GROUP BY country, user_id
    HAVING COUNT(*) >= 3
) qualified_users
GROUP BY country
ORDER BY country;
```

## Explanation

```
Step-by-Step Logic:

┌─────────────────────────────────────────────────────────────┐
│ Step 1: Filter to 2024 orders                               │
│ WHERE YEAR(order_date) = 2024                               │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│ Step 2: Group by country AND user_id                        │
│ Count orders and sum amounts per user per country           │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│ Step 3: Filter to users with >= 3 orders                    │
│ HAVING COUNT(*) >= 3                                        │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│ Step 4: Aggregate by country                                │
│ Count qualified users, sum their total amounts              │
└─────────────────────────────────────────────────────────────┘
```

## Sample Data & Expected Results

```
Sample Input:
| order_id | user_id | order_date | amount | country |
|----------|---------|------------|--------|---------|
| 1        | U1      | 2024-01-01 | 100    | USA     |
| 2        | U1      | 2024-02-01 | 150    | USA     |
| 3        | U1      | 2024-03-01 | 200    | USA     |
| 4        | U2      | 2024-01-15 | 50     | USA     |
| 5        | U3      | 2024-01-01 | 300    | UK      |
| 6        | U3      | 2024-02-01 | 400    | UK      |
| 7        | U3      | 2024-03-01 | 500    | UK      |

Expected Output:
| country | qualified_user_count | total_amount |
|---------|---------------------|--------------|
| UK      | 1                   | 1200         |
| USA     | 1                   | 450          |

Explanation:
- USA: U1 has 3 orders (qualified), U2 has 1 order (not qualified)
- UK: U3 has 3 orders (qualified)
```

---

